In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pickle

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/redfin-housing-market-data/zip_code_market_tracker.tsv000
/kaggle/input/redfin-housing-market-data/state_market_tracker.tsv000
/kaggle/input/redfin-housing-market-data/neighborhood_market_tracker.tsv000
/kaggle/input/redfin-housing-market-data/us_national_market_tracker.tsv000
/kaggle/input/redfin-housing-market-data/county_market_tracker.tsv000


In [11]:
# county data df
county_data = '/kaggle/input/redfin-housing-market-data/county_market_tracker.tsv000'
county_df = pd.read_csv(currentfile, sep='\t')
county_df

,period_begin,period_end,period_duration,region_type,region_type_id,table_id,is_seasonally_adjusted,region,city,state,...,sold_above_list_yoy,price_drops,price_drops_mom,price_drops_yoy,off_market_in_two_weeks,off_market_in_two_weeks_mom,off_market_in_two_weeks_yoy,parent_metro_region,parent_metro_region_metro_code,last_updated
0,2017-04-01,2017-04-30,30,county,5,170,f,"Morgan County, AL",NaN,Alabama,...,-0.069595,NaN,NaN,NaN,0.020833,0.020833,0.001225,"Decatur, AL",19460.0,2022-01-09 14:29:56
1,2014-12-01,2014-12-31,30,county,5,1457,f,"Hennepin County, MN",NaN,Minnesota,...,-0.098136,0.097744,-0.121909,-0.197220,0.172414,0.034483,-0.033469,"Minneapolis, MN",33460.0,2022-01-09 14:29:56
2,2018-02-01,2018-02-28,30,county,5,2319,f,"Tulsa County, OK",NaN,Oklahoma,...,-0.024840,0.206004,-0.026139,0.009313,0.479924,0.068277,-0.042568,"Tulsa, OK",46140.0,2022-01-09 14:29:56
3,2017-08-01,2017-08-31,30,county,5,1448,f,"Crow Wing County, MN",NaN,Minnesota,...,NaN,0.200000,-0.050000,NaN,NaN,NaN,NaN,"Brainerd, MN",14660.0,2022-01-09 14:29:56
4,2019-05-01,2019-05-31,30,county,5,1586,f,"Tate County, MS",NaN,Mississippi,...,0.107895,NaN,NaN,NaN,0.000000,-0.111111,-0.100000,"Memphis, TN",32820.0,2022-01-09 14:29:56
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
563117,2013-03-01,2013-03-31,30,county,5,2728,f,"Gonzales County, TX",NaN,Texas,...,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,Texas nonmetropolitan area,NaN,2022-01-09 14:29:56
563118,2013-09-01,2013-09-30,30,county,5,710,f,"Valley County, ID",NaN,Idaho,...,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,Idaho nonmetropolitan area,NaN,2022-01-09 14:29:56
563119,2016-06-01,2016-06-30,30,county,5,1261,f,"Livingston Parish, LA",NaN,Louisiana,...,-0.050180,0.189369,0.010930,0.028419,0.305785,-0.120571,0.020901,"Baton Rouge, LA",12940.0,2022-01-09 14:29:56
563120,2017-09-01,2017-09-30,30,county,5,2972,f,"Gloucester County, VA",NaN,Virginia,...,0.242424,NaN,NaN,NaN,0.000000,-0.111111,-0.166667,"Virginia Beach, VA",47260.0,2022-01-09 14:29:56


In [10]:
# state data df 
state = '/kaggle/input/redfin-housing-market-data/state_market_tracker.tsv000'
state_df = pd.read_csv(currentfile, sep='\t')
state_df

,period_begin,period_end,period_duration,region_type,region_type_id,table_id,is_seasonally_adjusted,region,city,state,...,sold_above_list_yoy,price_drops,price_drops_mom,price_drops_yoy,off_market_in_two_weeks,off_market_in_two_weeks_mom,off_market_in_two_weeks_yoy,parent_metro_region,parent_metro_region_metro_code,last_updated
0,2017-04-01,2017-04-30,30,county,5,170,f,"Morgan County, AL",NaN,Alabama,...,-0.069595,NaN,NaN,NaN,0.020833,0.020833,0.001225,"Decatur, AL",19460.0,2022-01-09 14:29:56
1,2014-12-01,2014-12-31,30,county,5,1457,f,"Hennepin County, MN",NaN,Minnesota,...,-0.098136,0.097744,-0.121909,-0.197220,0.172414,0.034483,-0.033469,"Minneapolis, MN",33460.0,2022-01-09 14:29:56
2,2018-02-01,2018-02-28,30,county,5,2319,f,"Tulsa County, OK",NaN,Oklahoma,...,-0.024840,0.206004,-0.026139,0.009313,0.479924,0.068277,-0.042568,"Tulsa, OK",46140.0,2022-01-09 14:29:56
3,2017-08-01,2017-08-31,30,county,5,1448,f,"Crow Wing County, MN",NaN,Minnesota,...,NaN,0.200000,-0.050000,NaN,NaN,NaN,NaN,"Brainerd, MN",14660.0,2022-01-09 14:29:56
4,2019-05-01,2019-05-31,30,county,5,1586,f,"Tate County, MS",NaN,Mississippi,...,0.107895,NaN,NaN,NaN,0.000000,-0.111111,-0.100000,"Memphis, TN",32820.0,2022-01-09 14:29:56
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
563117,2013-03-01,2013-03-31,30,county,5,2728,f,"Gonzales County, TX",NaN,Texas,...,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,Texas nonmetropolitan area,NaN,2022-01-09 14:29:56
563118,2013-09-01,2013-09-30,30,county,5,710,f,"Valley County, ID",NaN,Idaho,...,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,Idaho nonmetropolitan area,NaN,2022-01-09 14:29:56
563119,2016-06-01,2016-06-30,30,county,5,1261,f,"Livingston Parish, LA",NaN,Louisiana,...,-0.050180,0.189369,0.010930,0.028419,0.305785,-0.120571,0.020901,"Baton Rouge, LA",12940.0,2022-01-09 14:29:56
563120,2017-09-01,2017-09-30,30,county,5,2972,f,"Gloucester County, VA",NaN,Virginia,...,0.242424,NaN,NaN,NaN,0.000000,-0.111111,-0.166667,"Virginia Beach, VA",47260.0,2022-01-09 14:29:56
